In [ ]:
! pip install keras==2.2.5
! pip install tensorflow==1.14

In [2]:
import keras
keras.__version__

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

'2.2.5'

## Callback을 사용하여 모델의 훈련 과정 제어


Callback은 모델의 상태와 성능에 대한 모든 정보에 접근하고 훈련 중지, 모델 저장, 가중치 적재 또는 모델 상태 변경 등을 처리
- 모델의 fit() 메서드가 호출될 때 전달되는 객체
- 훈련하는 동안 모델은 여러 지점에서 callback을 호출


In [4]:
keras.callbacks.ModelCheckpoint
keras.callbacks.EarlyStopping
keras.callbacks.LearningRateScheduler
keras.callbacks.ReduceLROnPlateau
keras.callbacks.CSVLogger

keras.callbacks.CSVLogger

![Fig7-2-2](https://user-images.githubusercontent.com/44194558/142815090-7caee42c-5dfc-440e-a473-8a4bb7c4f652.PNG)

### ModelCheckpoint & EarlyStopping Callback

- EarlyStopping을 사용하면 정해진 에포크 동안 모니터링 지표가 향상되지 않을 때 훈련을 중지할 수 있음
- 일반적으로 이 callback 은 훈련하는 동안 모델을 계속 저장해 주는 ModelCheckpoint와 함께 사용
- 선택적으로 지금까지 가장 좋은 모델만 저장할 수 있음

참고 : https://deep-deep-deep.tistory.com/53

In [ ]:
import tensorflow.keras

callbacks_list = [
  keras.callbacks.EarlyStopping(
        monitor='acc',
        patience=1,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='my_model.h5',
        monitor='val_loss',
        save_best_only=True,
    )
]

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))

### ReduceLROnPlateau Callback

검증 손실이 향상되지 않을 때 학습률을 작게 할 수 있음.
- 손실 곡선이 평탄할 때 학습률을 작게 하거나 크게 하면 훈련 도중 지역 최솟값에서 효과적으로 빠져나올 수 있음.

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss'
        factor=0.1,
        patience=10,
    )
]

model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))

## Tensorboard


![Fig7-2-10](https://user-images.githubusercontent.com/44194558/142816027-b78adaac-8f77-43e1-94f6-b18d09901619.PNG)

In [5]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len,
                           name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])





Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_____________________________________________

In [10]:
# 텐서보드 callback 객체와 함께 훈련
# 지정된 위치에 로그 이벤트를 기록
import numpy as np

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='./my_log_dir',  # 로그 파일 저장 디렉토리
        histogram_freq=1,  # 1 에포크마다 활성화 출력의 히스토그램 기록
        embeddings_freq=1,  # 1에포크 마다 임베딩 데이터 기록
        embeddings_layer_names=['embed'],
        embeddings_data=x_test[:100],
    )
]
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 64s 3ms/step - loss: 0.4662 - acc: 0.8145 - val_loss: 0.4046 - val_acc: 0.8264
Epoch 2/10
20000/20000 [==============================] - 64s 3ms/step - loss: 0.4162 - acc: 0.7909 - val_loss: 0.4474 - val_acc: 0.8004
Epoch 3/10
20000/20000 [==============================] - 64s 3ms/step - loss: 0.3501 - acc: 0.7701 - val_loss: 0.5290 - val_acc: 0.7178
Epoch 4/10
20000/20000 [==============================] - 64s 3ms/step - loss: 0.3344 - acc: 0.7349 - val_loss: 0.5483 - val_acc: 0.7008
Epoch 5/10
20000/20000 [==============================] - 64s 3ms/step - loss: 0.2670 - acc: 0.7062 - val_loss: 0.6394 - val_acc: 0.6130
Epoch 6/10
20000/20000 [==============================] - 63s 3ms/step - loss: 0.2373 - acc: 0.6263 - val_loss: 0.7053 - val_acc: 0.5540
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Epoch 7/10
20000/20000 [=====================

In [11]:
!pip install tensorboard